In [1]:
import os
import sys
import ccxt
import pandas as pd

In [ ]:
def update_coin_data(coins):
    file = "" # fill this
    api = pd.read_csv(file)
    binance = ccxt.binance({
        'options': {'adjustForTimeDifference': True},
        'apiKey': api['apiKey'][0],
        'secret': api['secret'][0]
    })
    
    balance = binance.fetchBalance()    
    btc_price = float(binance.fetch_ticker('BTC/USDT')['info']['lastPrice'])
    df = pd.DataFrame(columns=['symbol', 'quantity', 'price', 'dollar_value'])
    
    for coin in coins:
        quantity = balance[coin]['total']
        if coin == 'BTC':
            price = btc_price
        else:
            btc_ratio = float(binance.fetch_ticker(coin + '/BTC')['info']['lastPrice'])
            price = btc_ratio * btc_price
        
        dollar_value = quantity * price
        df = df.append({'symbol': coin,'quantity':quantity,'price':price,'dollar_value':dollar_value}, ignore_index=True)
    
    df['weight'] = list(map(lambda x: x / df['dollar_value'].sum(), df['dollar_value']))
    df = df.sort_values('weight', ascending=False).reset_index(drop=True)
    return df

In [ ]:
def rebalance_order(coin1, coin2):
    try:
        binance.fetch_ticker(coin1 + '/' + coin2)['info']
        return coin1 + '/' + coin2, 'sell'
    except:
        try:
            binance.fetch_ticker(coin2 + '/' + coin1)['info']
            return coin2 + '/' + coin1, 'buy'
        except:            
            return coin1 + '/BTC', 'sell', coin2 + '/BTC', 'buy' # Note: could coin2 be 'BTC/' + coin/sell?

In [ ]:
def bnb_order(coin):
    # Note: is buy/sell correct?
    if balance['bnb']['total'] < 1:
        return
    try:
        ratio = binance.fetch_ticker(coin + '/BNB')['info']
        binance.create_order(coin + '/BNB', 'market', 'sell', quantity???)
    except:
        try:
            ratio = binance.fetch_ticker('BNB/' + coin)['info']
            binance.create_order('BNB/' + coin, 'market', 'buy???', quantity???)
        except:
            binance.create_order(coin + '/BTC', 'market', 'sell', quantity?) # get bnb to btc ratio? or coin/btc ratio?
            binance.create_order(?) # use BNB/BTC

In [ ]:
coins = ['BTC', 'ETH', 'XRP', 'OMG']
n = 1/len(coins)
thresh = .2 # percentage above average weight to trigger rebalance
data = update_coin_data(coins)

# Rebalance coins
while not data[data['weight'] > (n + thresh * n)].empty:
    order = rebalance_order(data['symbol'][0], data['symbol'][len(data) - 1])
    total_value = data['dollar_value'].sum()
    weight_to_move = (data['weight'][0] - data['weight'][len(data) - 1]) / 2
    quantity = round(weight_to_move * total_value / data[data['symbol'] == order[0][:3]]['price'].values[0], 5)
    binance.create_order(order[0], 'market', order[1], quantity)
    
    if len(order) > 2:
        quantity = round(weight_to_move * total_value / data[data['symbol'] == order[2][:3]]['price'].values[0], 5)
        binance.create_order(order[2], 'market', order[3], quantity)
        
# rebalance bnb
bnb_order(data['symbol'][0])
    

math proof-
1. 4 coins all 25%
2. Top coin goes to 35%, bottom two go to 22.5 and 17.5
3. Best to...

    a. take half of 10% gained, move the 5% to 17.5
    
    b. move all 10% gained to 17.5
    
    c. move 7.5 gained to 17.5, 2.5 to 22.5
    
